# Blog Generator Test Notebook

This notebook tests the functionality of the BlogGenerator class without importing it directly.

In [1]:
%reset -f
import os
import re
import random
from datetime import datetime
from dotenv import load_dotenv
from groq import Groq
from google_images_search import GoogleImagesSearch

## 1. Setup Environment and Paths

In [2]:
# Load environment variables
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
google_cx = os.getenv('GOOGLE_CX')

# Setup paths
website = "1"  # "1" for moroccoheritage, "2" for gobitcode
if website == "1":
    blog_path = "/home/adil/repo/morocco-heritage/data/blog/"
    image_path = "/home/adil/repo/morocco-heritage/public/static/images"
else:
    blog_path = "/home/adil/repo/gobitcode/data/blog/"
    image_path = "/home/adil/repo/gobitcode/public/static/images"

## 2. Helper Functions

In [60]:
def generate_slug(title):
    """Generate a URL-friendly slug from a title"""
    slug = title.lower()
    slug = re.sub(r'[^a-z0-9]+', '-', slug)
    return slug.strip('-')

def download_images(keyword, path, limit=1):
    """Download images from Google Images"""
    gis = GoogleImagesSearch(google_api_key, google_cx)
    
    search_params = {
        'q': keyword,
        'num': limit,
        'safe': 'off',
    }

    gis.search(search_params=search_params)
    image = gis.results()[0]
    image.download(path)
    
    # Rename the downloaded image
    old_image_name = os.path.basename(image.path)
    filename, extension = os.path.splitext(old_image_name)
    new_image_name = f"{generate_slug(filename)}{extension}"
    os.rename(image.path, os.path.join(path, new_image_name))
    
    return new_image_name

## 3. Blog Generation

In [ ]:
from utils import anycode_prompt, user_prompt
def generate_blog(seo_keywords):
    """Generate blog content using Groq API"""
    client = Groq(api_key=groq_api_key)
    
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "system",
                "content": anycode_prompt
            },
            {
                "role": "user",
                "content": user_prompt(seo_keywords)
            }
        ],
        temperature=1,
        max_tokens=1024*8,
        max_completion_tokens=1024*8,
        top_p=1,
        stream=False,
        stop=None,
    )

    return completion.choices[0].message.content

# Test parameters
seo_keywords = "Python Programming Basics"
system_prompt = "You are a technical writer creating blog posts about programming and technology."

print(f"Starting blog generation for topic: {seo_keywords}")

# Generate blog content
print("1. Generating blog content...")
mdx_blog = generate_blog(seo_keywords)
print("✓ Blog content generated")
print("length:", len(mdx_blog))
# llama 3311 # 

Starting blog generation for topic: Python Programming Basics
1. Generating blog content...
✓ Blog content generated
length: 3311


## open ai genearate 

In [ ]:
from openai import OpenAI
from utils import anycode_prompt, user_prompt




def generate_blog_openai(seo_keywords):
    """Generate blog content using OpenAI API"""
    
    client = OpenAI(api_key="")
    completion  = client.responses.create(
  model="gpt-4.1",
  input=[
    {
      "role": "system",
      "content": [
        {
          "type": "input_text",
          "text": anycode_prompt
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": user_prompt(seo_keywords)
        }
      ]
    }
  ],
  text={
    "format": {
      "type": "text"
    }
  },
  reasoning={},
  tools=[],
  temperature=1,
  max_output_tokens=2**63 -1,
  top_p=1,
  stream=False,
)
    return completion.output_text



# Test parameters
seo_keywords = "Python Programming Basics"
system_prompt = "You are a technical writer creating blog posts about programming and technology."


print(f"Starting blog generation for topic: {seo_keywords}")

# Generate blog content
print("1. Generating blog content...")
mdx_blog = generate_blog_openai(seo_keywords)
print("✓ Blog content generated")
print(mdx_blog[:50])
print("length:", len(mdx_blog))


Starting blog generation for topic: Python Programming Basics
1. Generating blog content...
✓ Blog content generated
```
---
title: Python Programming Basics: Your Gat
length: 3809


## 4. Blog Processing

In [62]:
def process_images(mdx_blog, image_folder, seo_keywords):
    """Process and download images for the blog"""
    img_tag_regex = r'<img[^>]*alt=\"([^\"]*)\"[^>]*>'
    matches = re.findall(img_tag_regex, mdx_blog)
    
    for match in matches:
        image_name = download_images(
            f"{seo_keywords} {match}",
            os.path.join(image_path, image_folder)
        )
        new_src = f'/static/images/{image_folder}/{image_name}'
        mdx_blog = re.sub(
            rf'(<img[^>]*src=\")[^\"]*(\"[^>]*alt=\"{re.escape(match)}\"[^>]*>)',
            rf'\1{new_src}\2',
            mdx_blog
        )
    
    return mdx_blog

def correct_title(mdx_blog):
    """Remove colon from the title line"""
    title_regex = r'^(title:.*?):(.*)$'
    return re.sub(title_regex, r'\1\2', mdx_blog, flags=re.MULTILINE)

def correct_date(mdx_blog):
    """Update the date to today's date"""
    today = datetime.now().strftime('%Y-%m-%d')
    date_regex = r'^(date:)\s*["\']?(\d{4}-\d{2}-\d{2})["\']?$'
    return re.sub(date_regex, rf"\1 '{today}'", mdx_blog, count=1, flags=re.MULTILINE)

In [71]:
test_blog = """
---
title: Mastering the Fundamentals: A B:"eginner's Guide to Python Programming Basics
date: '2025-04-13'
tags: ['Python', 'Programming Basics', 'Beginners Guide', 'Coding']
draft: false
summary: Learn the's esiisential concepts and building blocks of Python programming, including variables, data types, operators, control structures, and functions, to get started with coding in Python.
---

## Introduction

Python is a popular and versatile programming language that has become a staple in the world of computer science. With its simplicity, readability, and large community of developers, Python is an excellent language for beginners to learn. In this article, we'll delve into the fundamental concepts of Python programming, covering variables, data types, operators, control structures, and functions. By the end of this guide, you'll have a solid grasp of the basics and be ready to start building your own Python projects.

<img src="/static/images/python-logo.png" alt="Python logo" width="200" height="200" />

<TOCInline toc={props.toc} exclude="Introduction" />

## Variables and Data Types
"""



def correct_title(mdx_blog):
    """Remove ':', "'", and '"' when they appear after 'title:' or 'summary:'"""
    lines = mdx_blog.splitlines()
    new_lines = []
    for line in lines:
        if line.startswith("title:"):
            prefix, rest = line[:6], line[6:]
            rest = rest.replace(":", " ").replace("'", " ").replace('"', " ")
            line = prefix + rest
            print(rest)
        elif line.startswith("summary:"):
            prefix, rest = line[:8], line[8:]
            rest = rest.replace(":", " ").replace("'", " ").replace('"', " ")
            line = prefix + rest
            print(rest)
        new_lines.append(line)
    return "\n".join(new_lines)




print(correct_title(test_blog))

 Mastering the Fundamentals  A B  eginner s Guide to Python Programming Basics
 Learn the s esiisential concepts and building blocks of Python programming, including variables, data types, operators, control structures, and functions, to get started with coding in Python.

---
title: Mastering the Fundamentals  A B  eginner s Guide to Python Programming Basics
date: '2025-04-13'
tags: ['Python', 'Programming Basics', 'Beginners Guide', 'Coding']
draft: false
summary: Learn the s esiisential concepts and building blocks of Python programming, including variables, data types, operators, control structures, and functions, to get started with coding in Python.
---

## Introduction

Python is a popular and versatile programming language that has become a staple in the world of computer science. With its simplicity, readability, and large community of developers, Python is an excellent language for beginners to learn. In this article, we'll delve into the fundamental concepts of Python progr

## 5. Test the Complete Process

In [ ]:

# Generate slugs and folder
print("\n2. Processing blog metadata...")
title_match = re.search(r'^title:\s*(.*)$', mdx_blog, re.MULTILINE)
title = title_match.group(1)
slug = generate_slug(title)
random_suffix = random.randint(1000, 9999)
image_folder = f"{generate_slug(seo_keywords)}-{random_suffix}"
print(f"✓ Title: {title}")
print(f"✓ Generated slug: {slug}")
print(f"✓ Image folder: {image_folder}")



In [ ]:
# Process the blog
print("\n3. Processing blog content...")
mdx_blog = correct_title(mdx_blog)
mdx_blog = correct_date(mdx_blog)
mdx_blog = process_images(mdx_blog, image_folder, seo_keywords)
print("✓ Blog content processed")



In [ ]:
# Save the blog
print("\n4. Saving blog file...")
filename = f"{slug}.mdx"
blog_save_path = os.path.join(blog_path, filename)
with open(blog_save_path, 'w') as file:
    file.write(mdx_blog)
print(f"✓ Blog saved at: {blog_save_path}")

print("\nBlog generation completed successfully!")

## 6. Display Generated Content

In [ ]:
# Display the generated blog content
print(mdx_blog)

# Post on pinterest

In [6]:

import requests

headers = {
    'Authorization': 'Bearer ',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}

response = requests.get('https://api.pinterest.com/v5/boards', headers=headers)

if response.status_code == 200:
    boards = response.json()
    print(boards)
else:
    print(f"Failed to retrieve boards. Status code: {response.status_code}")
    print("Response:", response.json())
        

{'items': [{'id': '1136244249676952980', 'privacy': 'PUBLIC', 'collaborator_count': 0, 'pin_count': 15, 'media': {'pin_thumbnail_urls': ['https://i.pinimg.com/150x150/a3/ad/7a/a3ad7a11c207618401d1f2647ee0b5fe.jpg', 'https://i.pinimg.com/150x150/a6/32/68/a6326809f867974fdae4df33fc3e5c56.jpg', 'https://i.pinimg.com/150x150/41/91/94/419194fe85d942ac804533b6d1cd4dc4.jpg', 'https://i.pinimg.com/150x150/9e/6b/c9/9e6bc9be188ad54f268738a10a78d72a.jpg', 'https://i.pinimg.com/150x150/dc/0c/89/dc0c891dcbdc09cbe6154471b354ec69.jpg'], 'image_cover_url': 'https://i.pinimg.com/400x300/5b/76/52/5b7652f0f0243ef1775479d8ac367d7c.jpg'}, 'description': '', 'follower_count': 0, 'board_pins_modified_at': '2025-03-18T07:41:03.933000', 'name': 'Social', 'created_at': '2024-11-29T01:09:41', 'owner': {'username': 'anycodeit'}}], 'bookmark': None}


In [ ]:
from dotenv import load_dotenv
from pinterest.client import PinterestSDKClient
from pinterest.organic.pins import Pin
import os
import base64
import requests

def create_pinterest_pin(image_path, title, description, board_id):
    """
    Create a pin on Pinterest board using a local image file
    
    Args:
        image_path (str): Local path to the image file
        title (str): Title of the pin
        description (str): Description of the pin
        board_id (str): ID of the Pinterest board to pin to
        
    Returns:
        tuple: Pin ID and Pin Title if successful, (None, None) if failed
    """
    try:
        # Load environment variables
        load_dotenv()
        
        # Get client credentials
        client_id = os.getenv('PINTEREST_CLIENT_ID')
        client_secret = os.getenv('PINTEREST_CLIENT_SECRET')

        # Encode credentials for Basic Authentication
        credentials = f"{client_id}:{client_secret}"
        encoded_credentials = base64.b64encode(credentials.encode()).decode()

        # Set up headers and data for token request
        headers = {
            'Authorization': f'Basic {encoded_credentials}',
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        
        data = {
            'grant_type': 'client_credentials',
            'scope': 'pins:read,pins:write,boards:read,boards:write'
        }

        # Get access token
        response = requests.post('https://api.pinterest.com/v5/oauth/token', headers=headers, data=data)
        
        if response.status_code != 200:
            print(f"Failed to generate token. Status code: {response.status_code}")
            return None, None

        token_data = response.json()
        access_token = token_data['access_token']

        # Initialize Pinterest SDK Client
        PinterestSDKClient(access_token=access_token)

        # Read and encode image
        with open(image_path, "rb") as img_file:
            encoded_string = base64.b64encode(img_file.read()).decode()

        # Create pin
        pin_create = Pin.create(
            board_id=board_id,
            title=title,
            description=description,
            link="https://example.com",  # Replace with your link
            media_source={
                "source_type": "image_base64",
                "content_type": "image/jpeg",  # adjust to image/png if needed
                "data": encoded_string
            }
        )
        
        return pin_create.id, pin_create.title

    except Exception as e:
        print(f"Error creating pin: {str(e)}")
        return None, None

# Example usage
create_pinterest_pin(
    image_path='./format/big.jpg',
    title='My Local Image Pin',
    description='Pin from local file system',
    board_id='1136244249676952980'
)


('1136244180989407602', 'My Local Image Pin')